In [ ]:
from photonai.base import Hyperpipe, PipelineElement
from photonai_graph.GraphUtilities import get_random_connectivity_data, get_random_labels
from sklearn.model_selection import KFold
import pandas as pd
import os

In [ ]:
# make random matrices to simulate connectivity matrices
X = get_random_connectivity_data(number_of_nodes=12, number_of_individuals=100)
y = get_random_labels(l_type="classification", number_of_labels=100)

In [ ]:
# get toy coordinates
coord_dict = {'x': [39, 42, 42, 45, 3, 6, 6, 9, 42, 39, 45, 42],
              'y': [45, 39, 42, 42, 3, 3, 3, 3, 3, 3, 3, 3],
              'z': [45, 45, 39, 42, 42, 39, 45, 48, 3, 6, 6, 9]}
spatial_coords = pd.DataFrame(data=coord_dict)

# save test atlas to csv
path = "/tmp/test_coords.csv"
spatial_coords.to_csv(path, header=False, index=False)

In [ ]:
my_pipe = Hyperpipe('spatial_test_atlas_pipe',
                    inner_cv=KFold(n_splits=5),
                    outer_cv=KFold(n_splits=5),
                    optimizer='sk_opt',
                
                    optimizer_params={'n_configurations': 25},
                    metrics=['accuracy', 'balanced_accuracy', 'recall', 'precision'],
                    best_config_metric='accuracy')

my_pipe.add(PipelineElement('GraphConstructorSpatial', 
                            atlas_name='test', atlas_folder='/tmp/'))

my_pipe.add(PipelineElement('GraphEmbeddingLocallyLinearEmbedding'))

my_pipe.add(PipelineElement('SVC'))

In [ ]:
# fit pipe
my_pipe.fit(X, y)

In [ ]:
os.remove(path)